In [ ]:
%%javascript


# Abstract


In [ ]:
(refer 'sparqler.core)
(import java.net.URI
           [java.net URLEncoder URLDecoder])
kabob

In [ ]:
%%javascript
function loadCSS(filename){ 

       var file = document.createElement("link");
       file.setAttribute("rel", "stylesheet");
       file.setAttribute("type", "text/css");
       file.setAttribute("href", filename);
       document.head.appendChild(file);

}

//just call a function to load a new CSS:
loadCSS("http://cdn.jsdelivr.net/yasqe/2.11.6/yasqe.min.css");
loadCSS("http://yasr.yasgui.org/dist/yasr.min.css");
loadCSS("http://yasr.yasgui.org/doc/doc.min.css");

# Background

## AllegroGraph KaBOB Queries

### Basic Query

#### Basic Query in WebView

In [ ]:
(def simple-query "SELECT * WHERE {?sub ?pred ?obj}")
(def simple-url (str "http://amc-tantor.ucdenver.pvt:10035/repositories/kabob-dev#query/d/"
                (clojure.string/replace (URLEncoder/encode (boutros.matsu.util/pprint simple-query) "UTF-8") "+" "%20")))
(beakerx :set "query" simple-query)
(beakerx :set "url" simple-url)

In [ ]:
%%html
<iframe id="simple-iframe" src="http://amc-tantor.ucdenver.pvt:10035/repositories/kabob-dev#query" width = 100% height = 900></iframe> 

In [ ]:
%%javascript
document.getElementById('simple-iframe').src = beakerx.url;

#### Basic Query in YASQE

In [ ]:
%%html
<div id="agraph-simple" style="width: auto"></div>

In [ ]:
%%javascript
document.getElementById("agraph-simple").innerHTML = "";
var yasqe = YASQE(document.getElementById("agraph-simple"), 
                  {sparql: 
                   {showQueryButton: true,
                    value: beakerx.query,
                    endpoint: "http://amc-tantor.ucdenver.pvt:10035/repositories/kabob-dev/",
                    headers: { 
                      Authorization: 'secret:secret'
                    },
                    xhrFields: {
                      withCredentials: true
                    }}});

var yasr = YASR(document.getElementById("agraph-simple"), {
	//this way, the URLs in the results are prettified using the defined prefixes in the query
	getUsedPrefixes: yasqe.getPrefixesFromQuery
});

//link both together
yasqe.options.sparql.callbacks.complete = yasr.setResponse;

This doesn't quite work yet...

[Error] Failed to load resource: Preflight response is not successful (kabob-dev, line 0)

[Error] XMLHttpRequest cannot load http://amc-tantor.ucdenver.pvt:10035/repositories/kabob-dev/. Preflight response is not successful

### Complex Angiogenesis Query

#### The Query

TODO: Make an SPARQL Language plugin for Beaker

#### Webview Query

In [ ]:
%%html
<iframe id="angio-iframe" src="http://amc-tantor.ucdenver.pvt:10035/repositories/kabob-dev#query/183" width = 100% height = 900></iframe> 

Beaker doesn't have a SPARQL language yet, but it does have a SQL...

In [ ]:
%%sql
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX oboMeta: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX kiao: <http://kabob.ucdenver.edu/iao/>
PREFIX iaouniprot: <http://kabob.ucdenver.edu/iao/uniprot/>
PREFIX franzOption_clauseReorderer: <franz:identity>
PREFIX franzOption_chunkProcessingAllowed: <franz:yes>
PREFIX franzOption_memoryLimit: <franz:60g>
PREFIX franzOption_memoryExhaustionWarningPercentage: <franz:90.0>

# VEGF - angiogenesis GOBP/subclasses, the proteins that participate in those GOBPs, and their protein-protein interactions

select distinct ?angioSubs ?partProtId ?intProtId {

  #select angiogenesis GOBPs and its subclasses that are in the OBO namespace
  ?angioSubs rdfs:subClassOf* obo:GO_0001525 . #angiogenesis
  ?angioSubs oboMeta:hasOBONamespace ?OBONamespace . #OBO ns (BPs)

  #restriction - only want human proteins that participate in these GOBPs
  ?anonSubclass rdfs:subClassOf ?angioSubs .
  ?anonSubclass rdfs:subClassOf ?protRest .

  ?protRest rdf:type owl:Restriction .
  ?protRest owl:onProperty obo:RO_0000057 . #has_participant
  ?protRest owl:someValuesFrom ?protSub .

  ?protSub rdfs:subClassOf ?protClass .
  ?protClass rdfs:subClassOf* obo:CHEBI_36080 . #protein
  ?partProtId obo:IAO_0000219 ?protClass .

  ?protClass rdfs:subClassOf ?taxonRest .
  ?taxonRest owl:onProperty obo:RO_0002162 . #in taxon
  ?taxonRest owl:someValuesFrom obo:NCBITaxon_9606 . #human

  #swisprot ids – participating proteins
  ?idField obo:IAO_0000219 ?partProtId .
  ?idField kiao:hasTemplate iaouniprot:UniProtFileRecord_primaryAccessionDataField1 .
  ?record obo:BFO_0000051 ?idField .
  ?record obo:BFO_0000051 ?nameField .
  ?nameField kiao:hasTemplate iaouniprot:UniProtFileRecord_nameDataField1 .
  ?nameField obo:IAO_0000219 ?partProtName . # uniprot name

  #protein interactions
  ?protSub2 rdfs:subClassOf ?protClass .
  ?r_hasp owl:someValuesFrom ?protSub2 .
  ?r_hasp rdf:type owl:Restriction .
  ?r_hasp owl:onProperty obo:RO_0000057 . #has_participant

  ?interaction rdfs:subClassOf ?r_hasp .
  ?interaction rdfs:subClassOf ?rcard .
  ?rcard owl:cardinality 2 . # require 2 participants for a binary interaction
  ?rcard owl:onProperty obo:RO_0000057 . #has_participant

  ?interaction rdfs:subClassOf ?r_hasp2 .
  ?r_hasp2 rdf:type owl:Restriction .
  ?r_hasp2 owl:onProperty obo:RO_0000057 . #has_participant
  ?r_hasp2 owl:someValuesFrom ?protSub3 .

  ?protSub3 rdfs:subClassOf ?protClass3 .
  ?protClass3 rdfs:subClassOf ?taxonRest .
  ?protClass3 rdfs:subClassOf* obo:CHEBI_36080 . #protein
  ?intProtId obo:IAO_0000219 ?protClass3 .

  filter (?protSub3 != ?protSub2) .

  #swisprot ids – participating-protein’s protein interaction partner names
  ?idField2 obo:IAO_0000219 ?intProtId . #field denotes ice id
  ?idField2 kiao:hasTemplate iaouniprot:UniProtFileRecord_primaryAccessionDataField1 .
  ?record2 obo:BFO_0000051 ?idField2 . #record has part id field (column)
  ?record2 obo:BFO_0000051 ?nameField2 . #record has part name field
  ?nameField2 kiao:hasTemplate iaouniprot:UniProtFileRecord_nameDataField1 .
  ?nameField2 obo:IAO_0000219 ?intProtName . # uniprot name
  }

In [ ]:
(in-ns 'sparqler.core)

(defquery angiogenesis []
  
  (prefix "PREFIX franzOption_clauseReorderer: <franz:identity> PREFIX FranzOption_chunkProcessingAllowed: <franz:yes> PREFIX franzOption_memoryLimit: <franz:60g> PREFIX franzOption_memoryExhaustionWarningPercentage: <franz:90.0>")
  
  (select-distinct :angioSubs :partProtId :intProtId)
  
  (where
   
   (!# "select angiogenesis GOBPs and its subclasses that are in the OBO namespace")
   (of-biological-process :angioSubs GO:angiogenesis)
   
   (!# "restriction only want human proteins that participate in these GOBPs")
   (proteins-that-participates-in :protClass :angioSubs)
   
   (denotes :partProtID :protClass)
   
   (in-taxon obo:human :protClass)
   
   (!# "swisprot ids participating proteins")
   (swissprot-name :partProtId :partProtName)

   (!# "protein interactions")
   (with-interactions :protClass :intProtId)

   (!# "swisprot ids – participating-protein’s protein interaction partner names")
   (swissprot-name :intProtId :intProtName)
   ))


In [ ]:
(beakerx :set "angioquery" (angiogenesis))

In [ ]:
%%javascript
beakerx.angioquery

In [ ]:
%%html
<div id="yasqeangio" style="width: auto"></div>

In [ ]:
%%javascript
document.getElementById("yasqeangio").innerHTML = "";
var yasqeangio = YASQE(document.getElementById("yasqeangio"), 
                  {sparql: 
                   {showQueryButton: true,
                    value: beakerx.angioquery,
                    endpoint: "http://amc-tantor.ucdenver.pvt:10035/repositories/kabob-dev/",
                    headers: { 
                      Authorization: 'secret:secret'
                    },
                    xhrFields: {
                      withCredentials: true
                    }}});

yasqeangio.setValue(beakerx.angioquery);

var yasr = YASR(document.getElementById("agraph-simple"), {
	//this way, the URLs in the results are prettified using the defined prefixes in the query
	getUsedPrefixes: yasqe.getPrefixesFromQuery
});

//link both together
yasqeangio.options.sparql.callbacks.complete = yasr.setResponse;

# Methods

## SPARQLer

Manually composing SPARQL queries for KaBOB is tedious and error-prone.

In [ ]:
(def localize-cyc (localize uniprot:HUMAN_CYC))

In [ ]:
(kabob-query localize-cyc)

In [ ]:
(print-table result)

In [ ]:
(clojure.set/rename result {:protein_name "protein_name", :location_class "location_class", :location_name "location_name"}) 

In [ ]:
(use '[clojure.pprint :only [print-table]])
(def result (kabob-query q))
(println result)

## Now, with Javascript...

Now, lets look at it with a SPARQL javascript library.

In [ ]:
(def qu (QueryFactory/create q))
(QueryExecutionFactory/sparqlService (get kabob :endpoint) q) 2

In [ ]:
(beakerx :set "query" q)

In [ ]:
%%javascript
beakerx.query

In [ ]:
%%html
<div id="yasqe" style="width: auto"></div>

In [ ]:
%%javascript
document.getElementById("yasqeangio").innerHTML = "";
var yasqe = YASQE(document.getElementById("yasqe"), {
	sparql: {
		showQueryButton: true,
		endpoint: "http://amc-tantor.ucdenver.pvt:10035/repositories/kabob-dev/",
		headers: { 
			Authorization: 'secret:secret'
		},
		xhrFields: {
			withCredentials: true
		}
	}
});
yasqe.setValue(beakerx.query);

In [ ]:
(def qexec (doto (QueryExecutionFactory/sparqlService (get kabob :endpoint) q) (.setBasicAuthentication (get kabob :user) (char-array (get kabob :pass)))))
(def results (.execSelect qexec))

In [ ]:
(def outputStream (ByteArrayOutputStream.))
(ResultSetFormatter/outputAsJSON outputStream results)
(def json (String. (.toByteArray outputStream)))
(beakerx :set "result" json)

In [ ]:
%%html
<div id="yasr" style="width: auto"></div>

In [ ]:
%%javascript
var yasr = YASR(document.getElementById("yasr"));

yasr.setResponse(beakerx.result);

## Use d3sparql.js to visualize the result

### d3sparql website

In [ ]:
%%html
<iframe src="http://biohackathon.org/d3sparql/" width = 60% height = 1000></iframe> 

### d3sparql example

In [ ]:
%%html
    <div id="result"></div>

In [ ]:
%%javascript
json = JSON.parse(beakerx.result);
d3sparql

In [ ]:
%%javascript
 var config = {
        "key1":   "protein_name",       // SPARQL variable name for node1 (optional; default is the 1st variable)
        "key2":   "location_name",       // SPARQL variable name for node2 (optional; default is the 2nd varibale)
        "label1": "protein_name",  // SPARQL variable name for the label of node1 (optional; default is the 3rd variable)
        "label2": "location_name",  // SPARQL variable name for the label of node2 (optional; default is the 4th variable)
        //"value1": "node1value",  // SPARQL variable name for the value of node1 (optional; default is the 5th variable)
        //"value2": "node2value"   // SPARQL variable name for the value of node2 (optional; default is the 6th variable)
        //"radius": 100,
        "charge": -500,
        "distance": 200,
        "width": 1000,
        "height": 750,
        "selector": "#result"
      }
d3sparql.forcegraph(json, config)

In [ ]:
%%javascript
var config = {
        // for d3sparql.tree()
        "root": "protein_name",
        "parent": "protein_name",
        "child": "location_name",
        // for d3sparql.roundtree()
        "diameter": 800,
        "angle": 360,
        "depth": 200,
        "radius": 5,
        "selector": "#result"
      }
d3sparql.roundtree(json, config)

In [ ]:
%%javascript
var config = {
        "selector": "#result"
}
d3sparql.htmltable(json, config)

In [ ]:
%%javascript


In [ ]:
(def q (localize uniprot:HUMAN_CYC))
q

# Abstract